# Steup


In [1]:
# install git lfs
!sudo yum install -y amazon-linux-extras
!sudo amazon-linux-extras install epel -y
!sudo yum-config-manager --enable epel
!sudo yum install git-lfs -y

Loaded plugins: dkms-build-requires, extras_suggestions, langpacks, priorities,
              : update-motd, versionlock
amzn2-core                                               | 3.7 kB     00:00     
amzn2extra-docker                                        | 3.0 kB     00:00     
amzn2extra-kernel-5.10                                   | 3.0 kB     00:00     
amzn2extra-python3.8                                     | 3.0 kB     00:00     
centos-extras                                            | 2.9 kB     00:00     
copr:copr.fedorainfracloud.org:vbatts:shadow-utils-newxi | 3.3 kB     00:00     
https://download.docker.com/linux/centos/2/x86_64/stable/repodata/repomd.xml: [Errno 14] HTTPS Error 404 - Not Found
Trying other mirror.
libnvidia-container/x86_64/signature                     |  833 B     00:00     
libnvidia-container/x86_64/signature                     | 2.1 kB     00:00 !!! 
neuron                                                   | 2.9 kB     00:00     
neuron/prima

 72  collectd-python3         available    [ =stable ]
† Note on end-of-support. Use 'info' subcommand.
Loaded plugins: dkms-build-requires, extras_suggestions, langpacks, priorities,
              : update-motd, versionlock
================================== repo: epel ==================================
[epel]
async = True
bandwidth = 0
base_persistdir = /var/lib/yum/repos/x86_64/2
baseurl = 
cache = 0
cachedir = /var/cache/yum/x86_64/2/epel
check_config_file_age = True
compare_providers_priority = 80
cost = 1000
deltarpm_metadata_percentage = 100
deltarpm_percentage = 
enabled = True
enablegroups = True
exclude = 
failovermethod = priority
ftp_disable_epsv = False
gpgcadir = /var/lib/yum/repos/x86_64/2/epel/gpgcadir
gpgcakey = 
gpgcheck = True
gpgdir = /var/lib/yum/repos/x86_64/2/epel/gpgdir
gpgkey = file:///etc/pki/rpm-gpg/RPM-GPG-KEY-EPEL-7
hdrdir = /var/cache/yum/x86_64/2/epel/headers
http_caching = all
includepkgs = 
ip_resolve = 
keepalive = True
keepcache = False
mddownloadpoli

In [ ]:
import sagemaker
import boto3

sess = sagemaker.Session()
sagemaker_session_bucket = "bge-small-bucket"
iam_role_name = "arn:aws:iam::{account_id}:role/bge_sagemaker_instance_role"

try:
    aws_role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    aws_role= iam.get_role(RoleName=iam_role_name)['Role']['Arn']

aws_region = "us-west-2"

print(f"sagemaker role arn: {aws_role}")
print(f"sagemaker bucket: {sagemaker_session_bucket}")
print(f"sagemaker region: {aws_region}")

Couldn't call 'get_role' to get Role ARN from role name bge_sagemaker_instance_role to get Role path.


sagemaker role arn: arn:aws:iam::658349184098:role/bge_sagemaker_instance_role
sagemaker bucket: bge-small-bucket
sagemaker region: us-west-2


# Downloading model and uploading to s3


In [3]:
repository = "BAAI/bge-large-en-v1.5"
model_id=repository.split("/")[-1]
s3_location=f"s3://{sagemaker_session_bucket}/custom_inference/{model_id}/model.tar.gz"

In [5]:
## download the model from hf model
!git lfs install
!git clone https://huggingface.co/$repository

Git LFS initialized.
Cloning into 'bge-large-en-v1.5'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 36 (delta 12), reused 0 (delta 0), pack-reused 4
Unpacking objects: 100% (36/36), 344.15 KiB | 5.74 MiB/s, done.


In [6]:
# create model.tar.gz file will all model artifacts
!tar -czvf model.tar.gz -C bge-large-en-v1.5 .

./
./config_sentence_transformers.json
./.gitattributes
./pytorch_model.bin
./.ipynb_checkpoints/
./sentence_bert_config.json
./tokenizer_config.json
./modules.json
./tokenizer.json
./config.json
./code/
./code/requirements.txt
./code/inference.py
./.git/
./.git/branches/
./.git/description
./.git/index
./.git/config
./.git/objects/
./.git/objects/51/
./.git/objects/51/24f01766405bb68a800cb760882a6f179e4191
./.git/objects/74/
./.git/objects/74/a7e83d0715d74a7bce0744841d925a4724db46
./.git/objects/ab/
./.git/objects/ab/101ed8d012df683ef32f92cea51e97615ad07c
./.git/objects/e9/
./.git/objects/e9/570b1dae7c01f1b19f0dd1c02872f30f8a76fd
./.git/objects/pack/
./.git/objects/3e/
./.git/objects/3e/047a79957ad5c71040487bf1f9e74535d6eba6
./.git/objects/28/
./.git/objects/28/a25c67a05f50911962011834d45380db2be108
./.git/objects/68/
./.git/objects/68/8882a79f44442ddc1f60d70334a7ff5df0fb47
./.git/objects/fb/
./.git/objects/fb/140275c155a9c7c5a3b3e0e77a9e839594a938
./.git/objects/9b/
./.git/objects/9b

In [7]:
# # # upload to S3
!aws s3 cp model.tar.gz $s3_location

upload: ./model.tar.gz to s3://bge-small-bucket/custom_inference/bge-large-en-v1.5/model.tar.gz


# Deploying Endpoint (test)


In [8]:
from sagemaker.huggingface.model import HuggingFaceModel
from sagemaker.serverless import ServerlessInferenceConfig

In [9]:
# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data=s3_location,       # path to model and script
   role=iam_role_name,           # iam role with permissions to create an Endpoint
   transformers_version="4.26",  # transformers version used
   pytorch_version="1.13",       # pytorch version used
   py_version="py39",            # python version used
)

In [13]:
# #specifying memory size(MB) and MaxConcurrency
# serverless_config = ServerlessInferenceConfig(
#     memory_size_in_mb=3072,
#     max_concurrency=3
# )

# # deploy the serverless endpoint
# predictor = huggingface_model.deploy(
#     serverless_inference_config=serverless_config
# )
# deploy the endpoint endpoint
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g4dn.xlarge"
)

------------!

In [14]:
predictor.predict(["dummy text"])

{'embedding': '{"array": [[0.00699615478515625, -0.03204345703125, 0.041107177734375, 0.0207061767578125, -0.01322174072265625, -0.02813720703125, 0.045562744140625, 0.01009368896484375, -0.0015115737915039062, 0.02886962890625, 0.0302276611328125, 0.006732940673828125, -0.00916290283203125, -0.0027484893798828125, -0.0172882080078125, 0.005290985107421875, -0.0274200439453125, -0.033660888671875, -0.033294677734375, -0.026824951171875, -0.017913818359375, 0.04547119140625, -0.1083984375, -0.0390625, -0.03387451171875, -0.0168304443359375, -0.013397216796875, 0.008697509765625, 0.0313720703125, 0.05413818359375, 0.019683837890625, 0.0308685302734375, 0.0022068023681640625, -0.015594482421875, 0.005767822265625, -0.01122283935546875, 0.020416259765625, -0.02520751953125, -0.031494140625, -0.0188446044921875, -0.0002015829086303711, -0.00997161865234375, 0.029815673828125, -0.0146026611328125, -0.0635986328125, -0.02691650390625, -0.020233154296875, 0.001110076904296875, 0.02188110351562

In [17]:
def parse_streaming_body(data):
    # Decode the streaming body data and loads into dictionary
    data = json.loads(data.read().decode('utf-8'))
    
    # Extracting the embedded data
    return json.loads(data["embedding"])

In [18]:
import boto3
import json
runtime = boto3.Session().client("sagemaker-runtime")
embeddingEndpoint = "huggingface-pytorch-inference-2023-09-16-16-29-25-260"
input = ["dummy test"]
input_json = json.dumps(input)
response = runtime.invoke_endpoint(EndpointName=embeddingEndpoint, 
                                   ContentType="application/json",
                                   Body=input_json
                                  )
embedded_data = parse_streaming_body(response["Body"])

In [19]:
embedded_data

{'array': [[0.01267242431640625,
   0.01544952392578125,
   0.03076171875,
   0.0203704833984375,
   -0.04766845703125,
   -0.0201263427734375,
   0.027252197265625,
   0.02618408203125,
   -0.0162200927734375,
   0.0455322265625,
   0.040008544921875,
   0.00027632713317871094,
   0.0022258758544921875,
   -0.03265380859375,
   -0.02117919921875,
   0.0109405517578125,
   -0.040435791015625,
   -0.002590179443359375,
   -0.0237884521484375,
   -0.0343017578125,
   0.00719451904296875,
   0.0367431640625,
   -0.05908203125,
   -0.00878143310546875,
   -0.032379150390625,
   -0.01258087158203125,
   -0.005138397216796875,
   -0.00299835205078125,
   0.0266876220703125,
   0.040557861328125,
   0.012359619140625,
   0.0216217041015625,
   0.004608154296875,
   -0.0275726318359375,
   0.03021240234375,
   -0.031524658203125,
   0.049835205078125,
   -0.005558013916015625,
   -0.040435791015625,
   -0.038726806640625,
   -0.039306640625,
   0.0306243896484375,
   -0.005466461181640625,
   